In [1]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt
import GPy
import os
import matplotlib as mpl
import matplotlib.tri as tri
import ternary
import pickle
import datetime
from collections import Counter
import matplotlib.ticker as ticker
from sklearn import preprocessing
import pyDOE
import random
from scipy.stats import norm
import time
from sklearn.ensemble import RandomForestRegressor
import copy

# Load materials dataset

In [2]:
# go to directory where datasets reside
# load a dataset
dataset_name = './pool_bo_datasets/coe_opt'
raw_dataset = pd.read_csv(dataset_name + '_dataset_with_outliers.csv')
raw_dataset

,MSD,MaxDD,TI2_L,MATS4v,MATS4i,P_VSA_MR_5,P_VSA_MR_6,TDB06s,RDF040m,Mor20m,...,Mor10v,Mor20v,Mor25v,Mor26s,R7u,H-046,H-047,SHED_AL,ALOGP,log2mic
0,13.932682,31,11.445851,0.188084,0.258986,43.274814,159.839001,9.331435,7.298285,0.660512,...,0.330610,1.302476,0.677523,2.604714,0.832404,8,42,16.639784,4.7028,7
1,13.283292,29,7.552471,0.171382,0.113294,67.344865,184.958869,11.673967,19.789809,-0.455056,...,0.438021,0.522581,1.458045,1.054505,0.603622,16,60,16.387158,5.1054,7
2,14.610518,33,12.076614,0.043630,-0.052324,48.343878,159.839001,11.047232,12.123109,0.562753,...,0.289701,1.170651,0.910962,1.972913,1.433434,8,50,17.028720,4.7382,3
3,15.041094,35,10.110374,0.093130,0.153928,195.502995,195.649502,12.451389,14.213803,0.658169,...,0.622719,1.541754,1.230140,2.272437,0.861122,0,70,15.719735,-1.9444,5
4,13.021323,29,9.082050,0.221607,0.200261,36.930484,229.235009,13.480022,8.053732,1.143012,...,-0.127196,2.127560,1.399774,1.098055,0.383095,0,66,16.111429,1.1220,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,25.763577,53,18.888872,0.166512,0.119201,101.017297,403.976051,12.316947,27.588595,1.017754,...,-0.205759,2.660699,2.197656,0.737758,0.854541,24,108,7.752266,13.6948,10
119,15.041094,35,10.110374,0.092564,0.087114,124.538707,151.373362,11.769421,14.135271,0.577475,...,0.708059,1.545426,0.871095,3.473389,0.895211,20,58,15.047352,4.9926,8
120,14.108065,33,9.541178,-0.034516,0.176728,281.100296,137.790168,14.208487,6.034471,1.986645,...,2.064633,2.448180,0.960434,2.107053,0.869748,12,30,13.592465,6.4200,0
121,13.379485,30,8.487762,0.000425,-0.017312,83.301374,138.813428,10.885577,11.004007,0.222347,...,0.678675,0.948164,0.680881,2.319552,0.773280,12,45,15.840537,5.8295,7


In [3]:
feature_name = list(raw_dataset.columns)[:-1]
feature_name

['MSD',
 'MaxDD',
 'TI2_L',
 'MATS4v',
 'MATS4i',
 'P_VSA_MR_5',
 'P_VSA_MR_6',
 'TDB06s',
 'RDF040m',
 'Mor20m',
 'Mor25m',
 'Mor31m',
 'Mor10v',
 'Mor20v',
 'Mor25v',
 'Mor26s',
 'R7u',
 'H-046',
 'H-047',
 'SHED_AL',
 'ALOGP']

In [4]:
objective_name = list(raw_dataset.columns)[-1]
objective_name

'log2mic'

# Formulate optimization as global minimization

In [5]:
ds = copy.deepcopy(raw_dataset) 
ds[objective_name] = raw_dataset[objective_name].values
ds

,MSD,MaxDD,TI2_L,MATS4v,MATS4i,P_VSA_MR_5,P_VSA_MR_6,TDB06s,RDF040m,Mor20m,...,Mor10v,Mor20v,Mor25v,Mor26s,R7u,H-046,H-047,SHED_AL,ALOGP,log2mic
0,13.932682,31,11.445851,0.188084,0.258986,43.274814,159.839001,9.331435,7.298285,0.660512,...,0.330610,1.302476,0.677523,2.604714,0.832404,8,42,16.639784,4.7028,7
1,13.283292,29,7.552471,0.171382,0.113294,67.344865,184.958869,11.673967,19.789809,-0.455056,...,0.438021,0.522581,1.458045,1.054505,0.603622,16,60,16.387158,5.1054,7
2,14.610518,33,12.076614,0.043630,-0.052324,48.343878,159.839001,11.047232,12.123109,0.562753,...,0.289701,1.170651,0.910962,1.972913,1.433434,8,50,17.028720,4.7382,3
3,15.041094,35,10.110374,0.093130,0.153928,195.502995,195.649502,12.451389,14.213803,0.658169,...,0.622719,1.541754,1.230140,2.272437,0.861122,0,70,15.719735,-1.9444,5
4,13.021323,29,9.082050,0.221607,0.200261,36.930484,229.235009,13.480022,8.053732,1.143012,...,-0.127196,2.127560,1.399774,1.098055,0.383095,0,66,16.111429,1.1220,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,25.763577,53,18.888872,0.166512,0.119201,101.017297,403.976051,12.316947,27.588595,1.017754,...,-0.205759,2.660699,2.197656,0.737758,0.854541,24,108,7.752266,13.6948,10
119,15.041094,35,10.110374,0.092564,0.087114,124.538707,151.373362,11.769421,14.135271,0.577475,...,0.708059,1.545426,0.871095,3.473389,0.895211,20,58,15.047352,4.9926,8
120,14.108065,33,9.541178,-0.034516,0.176728,281.100296,137.790168,14.208487,6.034471,1.986645,...,2.064633,2.448180,0.960434,2.107053,0.869748,12,30,13.592465,6.4200,0
121,13.379485,30,8.487762,0.000425,-0.017312,83.301374,138.813428,10.885577,11.004007,0.222347,...,0.678675,0.948164,0.680881,2.319552,0.773280,12,45,15.840537,5.8295,7


# Process dataset for pool-based active learning

In [6]:
# for some datasets, each input feature x could have been evaluated more than once.
# to perform pool-based active learning, we need to group the data by unique input feature x value. 
# for each unique x in design space, we only keep the average of all evaluations there as its objective value
ds_grouped = ds.groupby(feature_name)[objective_name].agg(lambda x: x.unique().mean())
ds_grouped = ds_grouped.to_frame().reset_index()
ds_grouped

,MSD,MaxDD,TI2_L,MATS4v,MATS4i,P_VSA_MR_5,P_VSA_MR_6,TDB06s,RDF040m,Mor20m,...,Mor10v,Mor20v,Mor25v,Mor26s,R7u,H-046,H-047,SHED_AL,ALOGP,log2mic
0,10.528762,23,8.112883,0.161302,0.131739,23.534305,143.046247,12.197619,5.147189,0.568323,...,0.095305,1.014050,0.752885,0.598687,0.785891,0,36,12.602509,2.6360,7.0
1,10.923459,24,7.030634,0.189212,0.131110,35.301458,164.002558,12.559821,7.514137,0.441681,...,0.204084,1.101786,1.167664,0.258509,0.632414,0,48,12.921808,2.0459,7.0
2,10.970635,27,8.303276,-0.001008,0.015734,33.672432,109.460740,11.814393,5.225452,0.633673,...,0.157593,0.940806,0.664245,0.803437,0.790696,8,24,10.901384,3.6372,9.0
3,11.061649,26,8.592380,0.075619,0.053782,34.931056,146.333710,12.861753,7.268456,1.029013,...,0.027355,1.394814,0.967437,1.156418,0.742329,4,30,17.533911,5.0764,6.0
4,12.010400,31,8.906146,-0.111732,-0.031148,107.245806,117.133976,11.257597,6.428228,0.985166,...,1.419075,1.182500,0.670270,0.890274,0.967219,10,20,16.159173,7.2590,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,21.263186,49,12.841366,-0.032776,-0.111927,211.339014,205.596224,11.849128,30.651059,2.203380,...,2.079061,2.896488,2.374326,5.797578,1.260967,20,110,10.157195,7.1860,7.0
118,21.838325,49,13.041173,-0.028496,-0.102794,231.615268,205.596224,11.554938,39.133825,1.605585,...,2.333715,2.217253,2.768674,3.189573,1.233718,36,110,9.111141,10.8356,4.0
119,23.554047,55,13.703503,-0.044618,-0.131666,248.078093,205.596224,12.184462,33.565954,1.816205,...,2.290656,2.605375,2.531794,4.035097,1.180009,20,126,8.549829,6.6624,8.0
120,23.913646,55,13.781210,-0.039547,-0.121618,268.354347,205.596224,11.481233,34.043021,1.991467,...,1.974287,2.631367,2.366595,2.571878,1.130433,36,126,7.746233,10.3120,6.0


In [7]:
# these are the input feature x and objective value y used in framework
X_feature = ds_grouped[feature_name].values

y = np.array(ds_grouped[objective_name].values)

assert len(ds_grouped) == len(X_feature) == len(y)

# total number of data in set
N = len(ds_grouped)
print(N)

s_scaler = preprocessing.StandardScaler()


122


# Benchmarking Framework parameters

In [8]:
# here are some parameters of the framework
# number of ensembles. in the paper n_ensemble = 50.
n_ensemble = 50
# number of initial experiments
n_initial = 2
# number of top candidates, currently using top 5% of total dataset size
n_top = int(math.ceil(len(y) * 0.1))
# the top candidates and their indicies
top_indices = list(ds_grouped.sort_values(objective_name).head(n_top).index)

# random seeds used to distinguish between different ensembles
# there are 300 of them, but only first n_ensemble are used
seed_list = [4295, 8508, 326, 3135, 1549, 2528, 1274, 6545, 5971, 6269, 2422, 4287, 9320, 4932, 951, 4304, 1745, 5956, 7620, 4545, 6003, 9885, 5548, 9477, 30, 8992, 7559, 5034, 9071, 6437, 3389, 9816, 8617, 3712, 3626, 1660, 3309, 2427, 9872, 938, 5156, 7409, 7672, 3411, 3559, 9966, 7331, 8273, 8484, 5127, 2260, 6054, 5205, 311, 6056, 9456, 928, 6424, 7438, 8701, 8634, 4002, 6634, 8102, 8503, 1540, 9254, 7972, 7737, 3410, 4052, 8640, 9659, 8093, 7076, 7268, 2046, 7492, 3103, 3034, 7874, 5438, 4297, 291, 5436, 9021, 3711, 7837, 9188, 2036, 8013, 6188, 3734, 187, 1438, 1061, 674, 777, 7231, 7096, 3360, 4278, 5817, 5514, 3442, 6805, 6750, 8548, 9751, 3526, 9969, 8979, 1526, 1551, 2058, 6325, 1237, 5917, 5821, 9946, 5049, 654, 7750, 5149, 3545, 9165, 2837, 5621, 6501, 595, 3181, 1747, 4405, 4480, 4282, 9262, 6219, 3960, 4999, 1495, 6007, 9642, 3902, 3133, 1085, 3278, 1104, 5939, 7153, 971, 8733, 3785, 9056, 2020, 7249, 5021, 3384, 8740, 4593, 7869, 9941, 8813, 3688, 8139, 6436, 3742, 5503, 1587, 4766, 9846, 9117, 7001, 4853, 9346, 4927, 8480, 5298, 4753, 1151, 9768, 5405, 6196, 5721, 3419, 8090, 8166, 7834, 1480, 1150, 9002, 1134, 2237, 3995, 2029, 5336, 7050, 6857, 8794, 1754, 1184, 3558, 658, 6804, 8750, 5088, 1136, 626, 8462, 5203, 3196, 979, 7419, 1162, 5451, 6492, 1562, 8145, 8937, 8764, 4174, 7639, 8902, 7003, 765, 1554, 6135, 1689, 9530, 1398, 2273, 7925, 5948, 1036, 868, 4617, 1203, 7680, 7, 93, 3128, 5694, 6979, 7136, 8084, 5770, 9301, 1599, 737, 7018, 3774, 9843, 2296, 2287, 9875, 2349, 2469, 8941, 4973, 3798, 54, 2938, 4665, 3942, 3951, 9400, 3094, 2248, 3376, 1926, 5180, 1773, 3681, 1808, 350, 6669, 826, 539, 5313, 6193, 5752, 9370, 2782, 8399, 4881, 3166, 4906, 5829, 4827, 29, 6899, 9012, 6986, 4175, 1035, 8320, 7802, 3777, 6340, 7798, 7705]

# RF's surrogate models

In [9]:
# number of decision trees in Random Forest (RF) model
#n_est = 100

ho_opt = {'bootstrap': True, 'max_depth': 18, 'max_features': 'log2',
               'max_samples': 0.99, 'min_samples_leaf': 1,
               'min_samples_split': 2, 'n_estimators': 236}
n_est = 236

In [10]:
def RF_pred(X, RF_model):
    tree_predictions = []
    for j in np.arange(n_est):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]
    return mean, std


def EI(X, RF_model, y_best):

    mean, std = RF_pred(X, RF_model)
    
    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)

def LCB(X, RF_model, ratio):
    
    mean, std = RF_pred(X, RF_model)

    return - mean + ratio * std

def PI(X, RF_model, y_best):
    
    mean, std = RF_pred(X, RF_model)
    
    z = (y_best - mean)/std
    return norm.cdf(z)



# Pool-based active learning framework

In [11]:
# framework


# good practice to keep check of time used
start_time = time.time()

# these will carry results along optimization sequence from all n_ensemble runs
index_collection = []
X_collection = []
y_collection = []
TopCount_collection = []



for s in seed_list:
    
    if len(index_collection) == n_ensemble:
        break
    
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(N))
# index_learn is the pool of candidates to be examined
    index_learn = indices.copy()
# index_ is the list of candidates we have already observed
#     adding in the initial experiments
    index_ = random.sample(index_learn, n_initial)
    
#     list to store all observed good candidates' input feature X
    X_ = []
#     list to store all observed good candidates' objective value y
    y_ = []
#     number of top candidates found so far
    c = 0
#     list of cumulative number of top candidates found at each learning cycle
    TopCount_ = []
#     add the first n_initial experiments to collection
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        if i in top_indices:
            c += 1
        TopCount_.append(c)
        index_learn.remove(i)
           
    
#     for each of the the rest of (N - n_initial) learning cycles
#     this for loop ends when all candidates in pool are observed 
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        s_scaler = preprocessing.StandardScaler()
        X_train = X_
        y_train = [[i] for i in y_]
        
        
        RF_model = RandomForestRegressor(n_estimators= n_est, n_jobs= -1,
                                         bootstrap=ho_opt['bootstrap'], 
                                         max_depth=ho_opt['max_depth'], 
                                         max_features=ho_opt['max_features'], 
                                         max_samples=ho_opt['max_samples'],
                                         min_samples_leaf=ho_opt['min_samples_leaf'],
                                         min_samples_split=ho_opt['min_samples_split'])
        RF_model.fit(X_train, y_train)
        
        
#         by evaluating acquisition function values at candidates remaining in pool
#         we choose candidate with larger acquisition function value to be observed next   
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
#             #elect Acquisiton Function for BO
            ac_value = LCB(X_j, RF_model, 2)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        
        
        if next_index in top_indices:
            c += 1
        
        TopCount_.append(c)
        
        index_learn.remove(next_index)
        index_.append(next_index)        

    
    assert len(index_) == N
    
    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopCount_collection.append(TopCount_)
    
    
    print('Finished seed')
    
total_time = time.time() - start_time

master = np.array([index_collection, X_collection, y_collection, TopCount_collection, total_time])
#  name output file
np.save('pool_bo_coe_rf_lcb2_top010_with_outliers', master)
   

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed
initializing seed = 25
Finished seed
initializing seed = 26
Finished seed
initializin